In [2]:
import pandas as pd
import seaborn as sns 
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Data analyze

In [9]:
df = pd.read_parquet("./data/yellow_tripdata_2023-01.parquet")
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [10]:
len(df), len(df.columns)

(3066766, 19)

In [11]:
df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)

In [12]:
df["duration"] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds()/60)

In [14]:
df = df[df.duration > 0]

In [15]:
df.duration.describe(percentiles=[0.9,0.92,0.95,0.98])

count    3.065645e+06
mean     1.567475e+01
std      4.260107e+01
min      1.666667e-02
50%      1.153333e+01
90%      2.795000e+01
92%      3.055000e+01
95%      3.646667e+01
98%      4.873333e+01
max      1.002918e+04
Name: duration, dtype: float64

In [16]:
df = df[(df['duration'] >= 1) & (df["duration"] < 60)]
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,8.433333
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,6.316667
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,12.750000
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,9.616667
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,10.833333


# Feature selection

In [17]:
categorial = ["PULocationID", "DOLocationID"]
numerical = ["trip_distance"]

In [18]:
df[categorial] = df[categorial].astype(str)

In [19]:
train_dicts = df[categorial+numerical].to_dict(orient="records")

In [51]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [52]:
len(dv.feature_names_)

516

In [21]:
target = "duration"
y_train = df[target].values

In [22]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [23]:
y_pred = lr.predict(X_train)

In [24]:
mean_squared_error(y_train, y_pred, squared=False)

7.657848960483557

# Check accuarcy on February

In [44]:
df2 = pd.read_parquet("./data/yellow_tripdata_2023-02.parquet")
df2.tpep_dropoff_datetime = pd.to_datetime(df2.tpep_dropoff_datetime)
df2.tpep_pickup_datetime = pd.to_datetime(df2.tpep_pickup_datetime)

df2["duration"] = df2.tpep_dropoff_datetime - df2.tpep_pickup_datetime
df2.duration = df2.duration.apply(lambda td: td.total_seconds()/60)

In [45]:
df2 = df2[(df2['duration'] >= 1) & (df2["duration"] < 60)]

In [54]:
df2[categorial] = df2[categorial].astype(str)
train_dicts2 = df2[categorial+numerical].to_dict(orient="records")

In [55]:
X_val = dv.transform(train_dicts2)

In [56]:
y_val = df2[target].values
y_pred_val = lr.predict(X_val)

In [57]:
mean_squared_error(y_val, y_pred_val, squared=False)

7.818153505883039